In [23]:
import pandas as pd

# читаем основной и дополнительный датасеты
df_main = pd.read_csv("cian_flats.csv")
df_extra = pd.read_csv("cian_flats (4).csv")

# объединяем (добавляем строки второго к первому)
df_combined = pd.concat([df_main, df_extra], ignore_index=True)

# сохраняем обратно (можно перезаписать основной)
df_combined.to_csv("cian_flats.csv", index=False)

##1. Обработка датасета

импортируем нужные библиотеки все тип

In [22]:
import pandas as pd
import numpy as np